In [1]:
import tensorflow as tf
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# why dont these python scripts auto save in the directory..? 
    # Do I need to sync them or something? 
    

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import tensorflow.keras as keras
from tensorflow.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.backend import square, mean
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense
 
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

 
import datetime
from scipy.io import loadmat
import scipy
from scipy import stats


#4 covariate model with rain, n20, wfps, [and n2osf]
    # SL = 15, 30, 60  day sequences 
    # n2osf = 1, 3, 5, 7, 15 day sequences 
    # DO = 0.2,0.3,0.4   (0.5?)
    
# set some [LSTM model] variables 
SL="30"
n2osf="05"
DO="20"

n2osfI= int(n2osf)
SLI= int(SL)
DOI= int(DO)/100 

filenameX='/n2o_nn/LSTM_June2022/2_matlab_traintest_matdata/XTrain_base_SL'+SL+"_n2osf"+n2osf+".mat"
filenameY='/n2o_nn/LSTM_June2022/2_matlab_traintest_matdata/YTrain_base_SL'+SL+"_n2osf"+n2osf+".mat"
print(filenameX)

# load the mat data files     
matx = loadmat(filenameX) 
maty = loadmat(filenameY)

hdfname='/n2o_nn/LSTM_June2022/4_Python_LSTMmodels_hdf5/base_SL'+SL+"_n2osf"+n2osf+"_DO"+DO+".hdf5"
print(hdfname)
# potential other covariates that could be added:
# learningrate = lr001
# epoch = epoch500
# rmsprop = rmsprop



/n2o_nn/LSTM_June2022/2_matlab_traintest_matdata/XTrain_base_SL30_n2osf05.mat
/n2o_nn/LSTM_June2022/4_Python_LSTMmodels_hdf5/base_SL30_n2osf05_DO20.hdf5


In [4]:
print(tf.__version__)

1.15.0


In [5]:
# data prep for LSTM 

matx.keys()
maty.keys()
matx
dfx = pd.DataFrame(np.hstack((matx['XTrain1'])))
dfx.head
dfy = pd.DataFrame(np.hstack((maty['YTrain1'])))
dfy.head

arrx = dfx.values
arry = dfy.values
resultx = arrx[:,0]
resulty = arry[:,0]
 
xlist = []
ylist  = []

for i in range(len(resultx)):
    xlist.append(np.transpose(resultx[i]))
    ylist.append(np.transpose(resulty[i]))
    
Xtrain = np.asarray(xlist)
Xtrain 

#print( len(Xtrain) )
#print( len(xlist) )

Xtrain = np.asarray(xlist)

nvals=len(xlist)*SLI
print(nvals)
ynvals=len(ylist)*SLI

# this value is the rows/cells of the Xtrain * the SL, and # of covariates
Xtrain = np.reshape(Xtrain, (nvals, 4)) 
dfx = pd.DataFrame(Xtrain)

Ytrain = np.asarray(ylist).reshape(ynvals,1) 
dfy = pd.DataFrame(Ytrain)

dfx.columns = ['sampn2o', 'rainseq','wfpsseq','n2o']
df_targets = pd.DataFrame(dfy)
df_targets.columns = ['target n2o']

x_data = dfx.values
y_data = df_targets.values
num_data = len(x_data)
num_data 

#new training and testing for 30 day sequences and with lesser hidden neurons. Comment it out to go back to the original 60 day sequence batches
scaler = StandardScaler()
print(scaler.fit(y_data))
print(scaler.mean_)
y_data =scaler.transform(y_data)
y_data.shape

x_data = x_data.reshape((len(xlist),SLI,4))  #30 day 7 cov repeat after mistake , day since and fert applied + soil temp 10 with new hold out 
y_data = y_data.reshape((len(ylist),SLI,1))

x_data.shape, y_data.shape
 
    
#try standarsizing the target variable as the model weights were exploding due to the large variation in the y  or the target data


73770
StandardScaler()
[8.71197171]


((2459, 30, 4), (2459, 30, 1))

In [6]:
len(xlist)

2459

In [7]:
#LSTM architecture here  
    
# notes/architecture to QC 
    # changing the architecture to reduce hidden layer neurons to prevent overfitting with 25 neurons,
    # 50 precent dropout
    #and 30 time steps as input sequences. 
    
from keras.layers import Dropout

# from tensorflow.keras import layers 

model = Sequential()
# model = keras.Sequential() # tf2.3

# is this needed...? 
# # Add an Embedding layer expecting input vocab of size 1000, and
# # output embedding dimension of size 64.
# model.add(layers.Embedding(input_dim=1000, output_dim=64))

# for tensorflow 2.3
# https://www.tensorflow.org/guide/keras/rnn
# model.add(layers.LSTM(128,activation='tanh',
#                return_sequences=True,
#                ) )
# need to figure out the input_shape parameter 

# print(model)

# need to see about tensorflow syntax, AN used 1.15
#model.add(LSTM(50,activation='tanh',
#              return_sequences=True,  
#             input_shape=(30,7)))

model.add(LSTM(50,                  # what is this 50? layers I believe...
             activation='tanh',      # activation function 
             return_sequences=True,   
             input_shape=(SLI,4)))    # SL and covariates 

# this DO part needs to be vetted/tested ## 
  # if doing dropout 0.4 then need both 50 and 32 neuron 
    # otherwise dropout 0.2, 0.3 use just 50 layer 
  
    # if DOI=0.4, then add this second layer code line:
# model.add(LSTM(32, activation='tanh', return_sequences=True))

model.add(Dropout(DOI))    # set dropout layers 

##model.add(Dense(num_y_signals, activation='linear'))
model.add(Dense(1))
# # model.add(Dense(1, activation='relu'))
 
 
 # dont think below is needed...?:
# if False:
#     from tensorflow.python.keras.initializers import RandomUniform

#     # Maybe use lower init-ranges.
#     init = RandomUniform(minval=-0.05, maxval=0.05)

#     model.add(Dense(num_y_signals,
#                     activation='linear',
#                     kernel_initializer=init))
 
 





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
# optimizer = adam(lr=1e-2)
# optimizer
# from keras.optimizers import adam
from keras.optimizers import SGD
from keras.optimizers import RMSprop
# opt = adam(lr=0.001)

opt = RMSprop(lr=0.001)


In [9]:
model.compile(loss='mse', optimizer=opt)

In [10]:
# model.build()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 30, 50)            11000     
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 50)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 30, 1)             51        
Total params: 11,051
Trainable params: 11,051
Non-trainable params: 0
_________________________________________________________________


In [11]:
from keras.callbacks import ModelCheckpoint
path_checkpoint = '23_checkpoint.keras'
callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                      monitor='val_loss',
                                      verbose=1,
                                      save_weights_only=True,
                                      save_best_only=True)

In [12]:
callback_early_stopping = EarlyStopping(monitor='val_loss',
                                        patience=20, verbose=1) #initially patience was 5 for the 60 sequence model, now let's make it 20 to give the model more attempts at convergence 

In [13]:
callback_tensorboard = TensorBoard(log_dir='./23_logs/',
                                   histogram_freq=0,
                                   write_graph=False)

In [14]:
 callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                       factor=0.1,
                                       min_lr=1e-4,
                                       patience=20,
                                       verbose=1)

In [20]:
# callbacks = [callback_early_stopping,
#              callback_checkpoint,
#              callback_tensorboard,
#              callback_reduce_lr]

x_data.shape
y_data.shape

 
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min', restore_best_weights=True)

# mcp_save = ModelCheckpoint(filepath = '/Users/abhijnannath/crossvalid_18_30day_mdlwts_4covariate_base_Do20_lr001_epoch500_rmsprop_new_monitorloss_aug5.hdf5', save_best_only=True, monitor='loss', mode='min')
mcp_save = ModelCheckpoint(filepath = hdfname, save_best_only=True, monitor='loss', mode='min')
# mcp_save = ModelCheckpoint(filepath = '/n2o_nn/30day_7covariate_Do20_lr001_epoch500_rmsprop_new_monitorloss_n2osf7.hdf5', save_best_only=True, monitor='loss', mode='min')

reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, min_delta=1e-4, mode='min')
callbacks = [mcp_save]


In [21]:
# build the model 

# some old code commented out 
# from tensorflow.python.keras.utils.data_utils import Sequence 
# model.fit(x_batch,y_batch, epochs=1, steps_per_epoch=5, validation_data=validation_data)

# model.fit(x_batch,y_batch,
#           epochs=200,batch_size=16, validation_split=0.1, verbose=1)  #this is for the 60 day sequence. 

# history_fert_dayssince_4cov_lr001_rmsprop_lstm_dro20 = model.fit(x_data,y_data,
#          epochs=500,batch_size=16, validation_split=0.1, verbose=1, callbacks = callbacks)

lstm_mod = model.fit(x_data,y_data,
          epochs=500,batch_size=16, validation_split=0.1, verbose=1, callbacks = callbacks)

# is training/building (calibration) and testing (validation) done? 
    # holdout happens in the next script (estimation), but calibration/validation (train/test) should be done at the end of this script...
    

Train on 2213 samples, validate on 246 samples
Epoch 1/500
2213/2213 [==============================] - 1s 485us/step - loss: 0.5434 - val_loss: 0.0134
Epoch 2/500
2213/2213 [==============================] - 1s 520us/step - loss: 0.5345 - val_loss: 0.0117
Epoch 3/500
2213/2213 [==============================] - 1s 506us/step - loss: 0.5219 - val_loss: 0.0123
Epoch 4/500
2213/2213 [==============================] - 1s 515us/step - loss: 0.5026 - val_loss: 0.0108
Epoch 5/500
2213/2213 [==============================] - 1s 529us/step - loss: 0.4880 - val_loss: 0.0125
Epoch 6/500
2213/2213 [==============================] - 1s 532us/step - loss: 0.4805 - val_loss: 0.0108
Epoch 7/500
2213/2213 [==============================] - 1s 553us/step - loss: 0.4666 - val_loss: 0.0109
Epoch 8/500
2213/2213 [==============================] - 1s 554us/step - loss: 0.4531 - val_loss: 0.0110
Epoch 9/500
2213/2213 [==============================] - 1s 548us/step - loss: 0.4410 - val_loss: 0.0107
Epoch 10